<a href="https://colab.research.google.com/github/ahmedhesham47/Machine-Learning-Integrative-Framework-for-Predicting-ICB-Response/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Packages**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.pipeline import make_pipeline
from itertools import combinations
from functools import reduce
from sklearn.model_selection import GridSearchCV
import itertools
from scipy import stats
from sklearn.svm import SVC

## **Helper Functions**

In [3]:
'''
A function that pre-processes gene expression data.
It takes as an input a gene expression dataframe, some other parameter, and returns a filtered, pre-processed gene expression dataframe.
'''

def GeneExpressionPreprocessing(mRNAFile, DropList, GeneColumnName, Need_Transpose=True, Percent_To_FilterOut=20):
  for i in DropList: # Dropping columns
    mRNAFile = mRNAFile.drop(i, axis=1, errors='ignore')
  if Need_Transpose == True: # transpose the file such that genes are columns and samples are rows
    mRNAFile = mRNAFile.set_index(GeneColumnName)
    mRNAFile = mRNAFile.T
    mRNAFile = mRNAFile.rename_axis('Sample', axis=1)
  # Filter out genes which are zeroes in certain percentage from the data.
  filtered_data = mRNAFile.loc[:, (mRNAFile == 0).mean() * 100 <= Percent_To_FilterOut]
  return filtered_data

In [4]:
'''
A function that merges columns based on a common column.
It takes as an input a list of all the dataframes that should be merged and the merging column
This function is very important as we will use it often later
'''

def merge_on_common_column(dataframes, merge_column):
    merged_df = dataframes[0]
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on=merge_column)
    return merged_df

'''
A function that pre-processes clinical data.
It takes as an input a clinical dataframe, some other parameters, and returns a filtered, pre-processed clinical dataframe.
'''


def ClinicalPreProcessing(clinicalData, ColumnsToKeep, columnsToEncode, columnsToFillMean, categoricalVal="Yes"):
  clinical = clinicalData[ColumnsToKeep] # keeps only desired columns
  clinical = clinical.dropna(subset=[col for col in clinical.columns if col not in columnsToFillMean]) # drops columns that have nulls if they are not to be filled with the mean
  for i in columnsToEncode:
    clinical[i] = (clinical[i] == categoricalVal).astype(int)
    clinical[i] = clinical[i].astype('category') # encoding categorical data --> making sure that 1's and 0's, or similar data, are treated as categorical
  for i in columnsToFillMean:
    clinical[i] = pd.to_numeric(clinical[i], errors='coerce') # filling nulls with mean
    mean_col = clinical[i].mean()
    clinical[i].fillna(mean_col, inplace=True)

  return clinical

In [5]:
'''
A function that converts SNP mutation data (MAF files) to a binary matrix of 1's and 0's.
It takes as an input a mutation dataframe, a column for samples, and a column for genes. It returns a binary SNP mutation dataframe.
'''

def create_binary_matrix(data, sample_col, gene_col):
    unique_samples = data[sample_col].unique() # getting all unique samples
    unique_genes = data[gene_col].unique() # getting all unique genes
    binary_matrix = pd.DataFrame(0, index=unique_samples, columns=unique_genes) # creating an initial dataframe
    for _, row in data.iterrows():
        sample = row[sample_col]
        gene = row[gene_col]
        binary_matrix.at[sample, gene] = 1 # putting a 1 whenever a gene-sample combination has occured
    binary_matrix.reset_index(inplace=True)
    binary_matrix = binary_matrix.rename(columns={'index': 'Sample'}) # renaming index to Sample
    binary_matrix.columns = [col + '_SNP' for col in binary_matrix.columns] # this is needed for building an integrative model later where the same gene has expression, SNP mutation, and CNA mutation as features
    columns_to_convert = binary_matrix.columns
    for column in columns_to_convert:
      binary_matrix[column] = binary_matrix[column].astype('category')
    # May not be needed because typically classifiers are good at understanding that 1's and 0's only indicate categorical
    return binary_matrix

In [6]:
'''
A function that pre-processes CNA mutation data.
It takes as an input a CNA mutation dataframe, some other parameters, and returns a processed CNA mutation dataframe.
'''

def process_and_rename_columns(data, unique_col, cat=True):
    data_processed = data.drop_duplicates(subset=unique_col, keep='first') # Droppung duplicates
    data_processed.reset_index(drop=True, inplace=True)
    data_processed = data_processed.T # Transposing to be compatible with later analyses
    data_processed.columns = data_processed.iloc[0]
    data_processed = data_processed.drop(data_processed.index[0])
    data_processed.columns = [col + '_CN' for col in data_processed.columns] # this is needed for building an integrative model later where the same gene has expression, SNP mutation, and CNA mutation as features
    if cat: # converting to categorical if the numbers in the CNA dataframer indicate groups.
      columns_to_convert = data_processed.columns
      for column in columns_to_convert:
        data_processed[column] = data_processed[column].astype('category')
    return data_processed

In [7]:
'''
A function that merges more than two dataframes.
'''

def merge_dataframes(dataframeslist, commoncolumn):
  return reduce(lambda left, right: pd.merge(left, right, on=commoncolumn, how='inner'), dataframeslist)

In [8]:
'''
A function to get all best combinations from genes selected via feature selection methods.
It takes as an input a merged dataframe with all the selected genes and build all the combinations from 5 selection methods.
It returns "Final_selected_genes", which is a list of the genes coming from the combination that leads to the maximum number of genes.
'''

def Best_Combinations(merged):
  column_names = merged.columns
  column_combinations = combinations(column_names, 5)
  intersections = {}
  # Compute the intersection for each combination
  for combination in column_combinations:
      # Drop NaN values and find the intersections from that particular combination
      intersected_genes = set(merged[combination[0]].dropna()) & set(merged[combination[1]].dropna()) & \
                          set(merged[combination[2]].dropna()) & set(merged[combination[3]].dropna()) & \
                          set(merged[combination[4]].dropna())
      intersections[combination] = intersected_genes
  intersections_summary = {combo: len(genes) for combo, genes in intersections.items()} # this is not needed, it can only be used to see what each combination produced
  max_combination = max(intersections, key=lambda x: len(intersections[x])) # the combination leading to the maximum number of genes
  max_genes = intersections[max_combination] # the genes themselves
  Final_Selected_Genes = list(max_genes) # making sures they are in a list
  return Final_Selected_Genes

In [61]:
'''
A function to get a merged dataframe from all the feature selection methods
This is important because it will be passed to the function just explained above
It returns a merged dataframe.
'''

def get_merged_df(rf, fdr, cor, sfm, gb, rfe):
  #fdr.rename(columns={'Feature': 'Gene'}, inplace=True)

  rf_top_genes = rf['Feature']

  sfm_selected_genes = sfm['Feature']

  correlation_top_genes = cor['Feature']

  gb_top_genes = gb['Feature']

  fdr_top_genes = fdr['Feature']

  rfe_selected_genes = rfe['Feature']

  all_genes = set(rf_top_genes) | set(sfm_selected_genes) | set(correlation_top_genes) | set(gb_top_genes) | set(fdr_top_genes) | set(rfe_selected_genes)
  all_genes_list = list(all_genes)
  # In sets, the "|" means union, so we get all the genes from all the feature selection methods

  col = ['Random_Forest', 'Select_From_Model', 'Gene_Response_Correlation', 'Gradient_Boosting', 'Fisher_Discriminant_Ratio', 'Recursive_Feature_Elimination']
  merged_df = pd.DataFrame(index=all_genes_list, columns=col) # this is an empty dataframe with rows as ALL the genes and columns as the feature selection methods

  # We will now fill the dataframe with either a value (name of the gene) or NaN, which means the gene was not selected via this feature selection method
  # For example...
  # Random_Forest     Select_From_Model"
  # TP53              NaN
  # And so on...

  merged_df.loc[rf_top_genes, "Random_Forest"] = rf_top_genes.values
  merged_df.loc[sfm_selected_genes, "Select_From_Model"] = sfm_selected_genes.values
  merged_df.loc[correlation_top_genes, "Gene_Response_Correlation"] = correlation_top_genes.values
  merged_df.loc[gb_top_genes, "Gradient_Boosting"] = gb_top_genes.values
  merged_df.loc[fdr_top_genes, "Fisher_Discriminant_Ratio"] = fdr_top_genes.values
  merged_df.loc[rfe_selected_genes, "Recursive_Feature_Elimination"] = rfe_selected_genes.values
  return merged_df

## **Importing & PreProcessing cBioPortal Data**

In [ ]:
'''
Reading files from Ahmed's Drive
'''

# Aly
# clinical_patient = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/data_clinical_patient.txt', sep='\t')
# clinical_sample = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/data_clinical_sample.txt', sep='\t')
# data_cna = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/data_cna.txt', sep='\t')
# data_mrna = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/data_mrna_seq_tpm.txt', sep='\t')
# data_mutations = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/data_mutations.txt', sep='\t')
# CibersortData = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/CIBERSORTx.csv', sep=',')
# clinical_patient = clinical_patient.iloc[4:]
# clinical_sample = clinical_sample.iloc[4:]

In [10]:
'''
Reading files from Ahmed's Drive
'''

# Ahmed
clinical_patient = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/data_clinical_patient.txt', sep='\t')
clinical_sample = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/data_clinical_sample.txt', sep='\t')
data_cna = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/data_cna.txt', sep='\t')
data_mrna = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/data_mrna_seq_tpm.txt', sep='\t')
data_mutations = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/data_mutations.txt', sep='\t')
CibersortData = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/CIBERSORTx.csv', sep=',')
CibersortData = CibersortData.iloc[:, :-4]
clinical_patient = clinical_patient.iloc[4:]
clinical_sample = clinical_sample.iloc[4:]
CNA_Data = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/CNA_Data_Transformed.tsv', sep='\t')

In [ ]:
merged_clinical = merge_on_common_column([clinical_sample, clinical_patient], '#Patient Identifier')

# SPECIFIC TO THE FIRST DATA
# .............................................

# Solving issues in column names
immunotherapy_mapping = {
    'MK3475': 'MK3475',
    'MK 3475': 'MK3475',
    'Nivo': 'Nivolumab',
    'Nivolumab': 'Nivolumab',
    'nivolumab': 'Nivolumab',
    'Pembro': 'Pembrolizumab',
    'Pembrolizumab': 'Pembrolizumab',
    'pembrolizumab': 'Pembrolizumab',
}
merged_clinical['Immunotherapy'] = merged_clinical['Immunotherapy'].map(immunotherapy_mapping)
# Perform one-hot encoding for 'Immunotherapy' column
merged_clinical = pd.get_dummies(merged_clinical, columns=['Immunotherapy', 'Primary Type'])
# ..........................................................................

labels = merged_clinical[['Sample Identifier', 'Best Radiographic Response (RECIST 1.1)']]
response_mapping = {
    'Complete Response': 1,
    'Partial Response': 1,
    'Stable Disease': 0,
    'Progressive Disease': 0
}
labels['Best Radiographic Response (RECIST 1.1)'] = labels['Best Radiographic Response (RECIST 1.1)'].map(response_mapping)
labels = labels.dropna(subset = ['Best Radiographic Response (RECIST 1.1)', 'Sample Identifier'])
labels = labels.rename(columns={'Best Radiographic Response (RECIST 1.1)': 'ICB Response'})
labels['ICB Response'] = labels['ICB Response'].astype('category')
filter_ids = labels['Sample Identifier']

# Preparing all the parameters needed for pre-processing cBioportal clinical data

columnsToFillNANWithMean = ['LDH (treatment Start)']
ImportantClinicalFeatures = ['Sample Identifier', '#Patient Identifier', 'Prior CTLA4', 'Num Prior Therapies', 'LDH (treatment Start)', 'Total Mutations',
       'Nonsynonymous Mutation Count', 'Mutations Clonal', 'Mutations Subclonal', 'Heterogeneity', 'Total Neoantigen', 'CNA Prop', 'Purity', 'Ploidy',
       'Brain Metastasis', 'Cutaneous Subcutaneous Metastasis', 'LN Metastasis', 'Lung Metastasis', 'Liver Visceral Metastasis', 'Bone Metastasis',
       'UV-Induced Mutations', 'Alkylating Chemotherapy', 'Immunotherapy_MK3475',
       'Immunotherapy_Nivolumab', 'Immunotherapy_Pembrolizumab', 'Primary Type_acral', 'Primary Type_mucosal', 'Primary Type_occult',
       'Primary Type_skin']
FeaturesNeedEncoding = ['Prior CTLA4', 'Brain Metastasis', 'Cutaneous Subcutaneous Metastasis', 'LN Metastasis', 'Lung Metastasis', 'Liver Visceral Metastasis', 'Bone Metastasis']
ProcessedClinicalData = ClinicalPreProcessing(merged_clinical, ImportantClinicalFeatures, FeaturesNeedEncoding, columnsToFillNANWithMean)
ProcessedClinicalData = ProcessedClinicalData.drop('#Patient Identifier', axis=1)
ProcessedClinicalData = ProcessedClinicalData[ProcessedClinicalData['Sample Identifier'].isin(filter_ids)]

In [14]:
ProcessedClinicalData

,Sample Identifier,Prior CTLA4,Num Prior Therapies,LDH (treatment Start),Total Mutations,Nonsynonymous Mutation Count,Mutations Clonal,Mutations Subclonal,Heterogeneity,Total Neoantigen,...,Bone Metastasis,UV-Induced Mutations,Alkylating Chemotherapy,Immunotherapy_MK3475,Immunotherapy_Nivolumab,Immunotherapy_Pembrolizumab,Primary Type_acral,Primary Type_mucosal,Primary Type_occult,Primary Type_skin
0,Sample1,1,1,323.269504,34,22,12,10,0.454545455,49,...,1,0.50942587,2.51455596,1,0,0,0,0,1,0
1,Sample10,1,2,2186.000000,96,71,48,22,0.314285714,230,...,1,35.8833464,6.81073463,1,0,0,0,0,0,1
2,Sample100,1,1,313.000000,200,126,98,24,0.196721311,301,...,0,134.899883,12.6984645,0,1,0,0,0,1,0
4,Sample104,0,0,270.000000,130,96,65,28,0.301075269,329,...,0,15.6698071,3.58768501,1,0,0,0,1,0,0
6,Sample106,0,1,403.000000,380,245,194,36,0.156521739,726,...,0,284.58223,27.3833807,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample9,0,0,192.000000,911,652,562,80,0.124610592,1996,...,0,637.426283,36.3703376,0,1,0,0,0,0,1
140,Sample94,1,2,817.000000,414,268,227,21,0.084677419,929,...,1,339.824541,2.28E-08,1,0,0,0,0,0,1
141,Sample96,0,0,174.000000,959,649,588,50,0.078369906,2136,...,0,851.443224,2.37349696,0,1,0,0,0,0,1
142,Sample98,0,0,192.000000,472,302,219,74,0.252559727,790,...,0,391.630271,6.51309681,0,1,0,0,0,1,0


In [15]:
labels

,Sample Identifier,ICB Response
0,Sample1,0.0
1,Sample10,1.0
2,Sample100,1.0
4,Sample104,1.0
6,Sample106,1.0
...,...,...
139,Sample9,0.0
140,Sample94,0.0
141,Sample96,1.0
142,Sample98,0.0


In [16]:
GeneExpressionData = GeneExpressionPreprocessing(data_mrna, ['Entrez_Gene_Id'], 'Hugo_Symbol')
GeneExpressionData = GeneExpressionData.reset_index()
GeneExpressionData = GeneExpressionData.rename(columns={'index': 'Sample Identifier'})
GeneExpressionData = GeneExpressionData[GeneExpressionData['Sample Identifier'].isin(filter_ids)]

MutationData = create_binary_matrix(data_mutations, 'Tumor_Sample_Barcode', 'Hugo_Symbol')
MutationData = MutationData.rename(columns={'Sample_SNP': 'Sample Identifier'})
MutationData = MutationData[MutationData['Sample Identifier'].isin(filter_ids)]

# Preprocessing step done only once and commented because it takes so much time
# We saved the resulting file and we directly read it above as CNA_Data

# CNA_Data = process_and_rename_columns(data_cna, 'Hugo_Symbol')
# CNA_Data = CNA_Data.reset_index()
# CNA_Data = CNA_Data.rename(columns={'index': 'Sample Identifier'})

CNA_Data = CNA_Data[CNA_Data['Sample Identifier'].isin(filter_ids)]

In [17]:
GeneExpressionData

Sample,Sample Identifier,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A4GALT,A4GNT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,Sample100,19.614242,0.826789,504.956774,0.844380,0.228686,8.215113,0.844380,48.340750,7.740149,...,7.230003,23.818550,10.695479,3.817301,3.008103,12.964750,14.143363,204.832492,58.438126,12.947159
3,Sample106,18.363130,0.902715,93.811126,0.095023,0.118778,0.427602,0.118778,33.329200,5.582582,...,11.616521,11.664032,11.022629,1.900453,3.159504,8.528284,19.432135,42.593910,30.169696,17.888017
4,Sample107,0.000000,0.000000,540.552469,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,67.725304,0.000000,0.000000,0.000000,0.000000,0.000000,377.534480,19.590340,0.000000
5,Sample108,0.606043,0.000000,696.342904,0.000000,0.000000,0.000000,0.000000,18.181277,0.000000,...,4.318053,54.013543,0.000000,0.000000,0.454532,13.029915,7.575532,64.695043,45.225926,5.984670
6,Sample10,10.049930,0.720561,120.257841,0.151697,0.132735,1.365274,0.075849,69.041123,6.845330,...,14.714614,47.575989,25.409257,0.910182,4.171669,10.391248,20.573913,71.676859,17.862328,14.487069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Sample196,24.699800,1.678815,697.036074,0.289451,0.135077,0.154374,0.231561,66.670163,3.627783,...,5.731125,20.589599,20.010697,3.859344,3.647080,12.330603,18.100322,77.379842,27.343450,18.119619
117,Sample197,70.098366,1.780975,127.249646,0.000000,0.000000,1.280701,0.300164,44.544380,10.966002,...,13.347305,18.149934,22.932551,1.800986,5.823187,14.628006,28.675695,61.413613,20.131018,22.772464
118,Sample204,10.518109,1.887154,199.455539,0.333027,8.408937,1.415366,0.388532,40.074273,13.570858,...,6.216508,13.598611,13.903885,3.274767,4.024079,7.243342,25.559838,79.066207,27.613505,17.428423
119,Sample205,21.005952,2.471288,1371.668063,0.205941,0.128713,0.900991,0.308911,52.437652,25.253479,...,7.877232,29.269323,11.609907,2.651487,2.265348,8.906935,23.605953,109.740653,28.934669,13.051492


In [18]:
MutationData

,Sample Identifier,LRP1B_SNP,EIF3H_SNP,C12orf55_SNP,CLEC18B_SNP,KIAA1045_SNP,BCLAF1_SNP,MUC16_SNP,RAB11FIP1_SNP,ZNF276_SNP,...,AC079922.2_SNP,PFN1P2_SNP,MT-TS1_SNP,MT-ND5_SNP,RNU6-1154P_SNP,PRPF38AP2_SNP,PABPC1P5_SNP,TMCO1_SNP,DNAJC19P9_SNP,PROK2_SNP
1,Sample203,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sample159,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sample108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sample104,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
5,Sample21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample180,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
140,Sample121,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
141,Sample72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,Sample59,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
CNA_Data

,Sample Identifier,5S_rRNA_CN,7SK_CN,A1BG_CN,A1BG-AS1_CN,A1CF_CN,A2M_CN,A2M-AS1_CN,A2ML1_CN,A2ML1-AS1_CN,...,ZUFSP_CN,ZW10_CN,ZWILCH_CN,ZWINT_CN,ZXDC_CN,ZYG11A_CN,ZYG11B_CN,ZYX_CN,ZZEF1_CN,ZZZ3_CN
0,Sample1,-1,0,0,0,-1,0,0,0,0,...,0,0,-1,-1,-1,-1,-1,0,0,-1
1,Sample10,0,-1,0,0,-1,0,0,0,0,...,0,0,0,-1,0,-1,-1,0,-1,-1
2,Sample100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sample104,0,0,0,0,-1,0,0,0,0,...,-1,0,0,-1,0,0,0,0,0,0
6,Sample106,0,0,0,0,-1,0,0,0,0,...,0,0,0,-1,-1,-1,-1,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample9,-1,0,0,0,0,0,0,0,0,...,0,-1,0,0,0,0,0,0,0,0
140,Sample94,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
141,Sample96,0,0,0,0,-1,0,0,0,0,...,-1,-1,0,-1,0,0,0,0,-1,0
142,Sample98,0,0,0,0,-1,0,0,0,0,...,0,-1,0,-1,-1,-1,-1,0,0,-1


## **Importing & PreProcessing Ravi Data**
Very similar workflow to cBioPortal Data

In [ ]:
# Aly
# clinical1 = pd.read_excel('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/clinical.xlsx')
# clinical2 = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/antigen_presentation.csv', sep='\t')
# clinical3 = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/purity_ploidy.csv', sep='\t')
# clinical4 = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/total_amps_dels.csv', sep='\t')
# clinical5 = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/tmb.csv', sep='\t')
# limma = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/DEGs.csv', sep='\t')
# Limma_Genes = list(limma['hgnc_symbol'].values)
# data_cna = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/gistic_gene.csv', sep='\t')
# data_mrna = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/mrna_tpm.csv', sep='\t')
# data_mutations = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/Absolute_MAF_Final.csv', sep='\t')
# CibersortData = pd.read_csv('/content/drive/MyDrive/Machine Learning/Machine Learning Project/New Data/CIBERSORT_FINAL_GOOD_DATA.csv', sep=',')
# CibersortData = CibersortData.iloc[:, :-4]


In [24]:
# Ahmed
clinical1 = pd.read_excel('/content/drive/MyDrive/Machine Learning Project/New Data/clinical.xlsx')
clinical2 = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/antigen_presentation.csv', sep='\t')
clinical3 = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/purity_ploidy.csv', sep='\t')
clinical4 = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/total_amps_dels.csv', sep='\t')
clinical5 = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/tmb.csv', sep='\t')
limma = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/DEGs.csv', sep='\t')
Limma_Genes = list(limma['hgnc_symbol'].values)
data_cna = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/gistic_gene.csv', sep='\t')
data_mrna = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/mrna_tpm.csv', sep='\t')
data_mutations = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/Absolute_MAF_Final.csv', sep='\t')
CibersortData = pd.read_csv('/content/drive/MyDrive/Machine Learning Project/New Data/CIBERSORT_FINAL_GOOD_DATA.csv', sep=',')
CibersortData = CibersortData.iloc[:, :-4]

In [25]:
clinical1['Harmonized_SU2C_WES_Tumor_Sample_ID_v2'] = clinical1['Harmonized_SU2C_WES_Tumor_Sample_ID_v2'].str.replace('-T', '')
merged_clinical = merge_on_common_column([clinical1, clinical2, clinical3, clinical4, clinical5], 'Harmonized_SU2C_WES_Tumor_Sample_ID_v2')
merged_clinical = pd.get_dummies(merged_clinical, columns=['Agent_PD1_Category', 'Agent_PD1'])

labels = clinical1[['Harmonized_SU2C_WES_Tumor_Sample_ID_v2', 'Harmonized_Confirmed_BOR']]
labels = labels.rename(columns={'Harmonized_SU2C_WES_Tumor_Sample_ID_v2': 'Sample Identifier'})

response_mapping = {
    'CR': 1,
    'PR': 1,
    'SD': 0,
    'PD': 0
}
labels['Harmonized_Confirmed_BOR'] = labels['Harmonized_Confirmed_BOR'].map(response_mapping)
labels = labels.dropna(subset = ['Harmonized_Confirmed_BOR', 'Sample Identifier'])
labels = labels.rename(columns={'Harmonized_Confirmed_BOR': 'ICB Response'})
labels['ICB Response'] = labels['ICB Response'].astype('category')
filter_ids = labels['Sample Identifier']

In [28]:
columnsToFillNANWithMean = ['Patient_Smoking_Pack_Years_Harmonized']
ImportantClinicalFeatures = ['Harmonized_SU2C_WES_Tumor_Sample_ID_v2', 'Patient_Smoking_Pack_Years_Harmonized', 'Initial_Stage',
'Agent_PD1_Category_PD(L)1',
       'Agent_PD1_Category_PD(L)1 + CTLA4',
       'Agent_PD1_Category_PD(L)1 + Other', 'Agent_PD1_Atezolizumab',
       'Agent_PD1_Avelumab', 'Agent_PD1_Nivolumab',
       'Agent_PD1_Nivolumab + Ipilimumab', 'Agent_PD1_Nivolumab + Lirilumab',
       'Agent_PD1_Pembrolizumab', 'Prior_TKI', 'HLA_LOH_present', 'HLA_hom_present', 'B2M_altered', 'Ploidy', 'Purity', 'Total_amps',
       'Total_dels', 'TMB', 'TMB_clonal', 'TMB_subclonal', 'TMB_indel',
       'Neoantigens', 'Neoantigens_clonal', 'Neoantigens_subclonal']
FeaturesNeedEncoding = []
ProcessedClinicalData = ClinicalPreProcessing(merged_clinical, ImportantClinicalFeatures, FeaturesNeedEncoding, columnsToFillNANWithMean)
ProcessedClinicalData = ProcessedClinicalData.rename(columns={'Harmonized_SU2C_WES_Tumor_Sample_ID_v2': 'Sample Identifier'})
ProcessedClinicalData = ProcessedClinicalData[ProcessedClinicalData['Sample Identifier'].isin(filter_ids)]

In [29]:
ProcessedClinicalData

,Sample Identifier,Patient_Smoking_Pack_Years_Harmonized,Initial_Stage,Agent_PD1_Category_PD(L)1,Agent_PD1_Category_PD(L)1 + CTLA4,Agent_PD1_Category_PD(L)1 + Other,Agent_PD1_Atezolizumab,Agent_PD1_Avelumab,Agent_PD1_Nivolumab,Agent_PD1_Nivolumab + Ipilimumab,...,Purity,Total_amps,Total_dels,TMB,TMB_clonal,TMB_subclonal,TMB_indel,Neoantigens,Neoantigens_clonal,Neoantigens_subclonal
0,SU2CLC-CLE-NIVO11,40.0,3.0,1,0,0,0,0,1,0,...,0.57,17.0,17.0,1006,217,713,3.0,154,25,116
1,SU2CLC-CLE-NIVO181,0.0,2.0,1,0,0,0,0,1,0,...,0.25,20.0,15.0,149,133,0,5.0,38,35,0
2,SU2CLC-CLE-NIVO191,15.0,4.0,1,0,0,0,0,1,0,...,0.37,10.0,10.0,602,535,15,10.0,125,110,8
3,SU2CLC-CLE-NIVO31,0.0,4.0,1,0,0,0,0,1,0,...,0.93,18.0,16.0,84,44,36,2.0,6,1,5
4,SU2CLC-CLE-NIVO41,30.0,1.0,1,0,0,0,0,1,0,...,0.62,21.0,18.0,172,106,65,0.0,24,14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-25231,40.0,3.0,1,0,0,1,0,0,0,...,0.59,23.0,20.0,711,455,240,18.0,79,49,29
305,SU2CLC-YAL-25251,40.0,3.0,1,0,0,0,0,1,0,...,0.48,22.0,19.0,576,454,105,8.0,121,92,23
306,SU2CLC-YAL-25321,25.0,4.0,0,1,0,0,0,0,1,...,0.25,5.0,17.0,780,703,52,8.0,169,146,18
307,SU2CLC-YAL-25371,12.0,4.0,1,0,0,0,0,0,0,...,0.28,8.0,11.0,291,197,69,5.0,98,59,30


In [30]:
GeneExpressionData = GeneExpressionPreprocessing(data_mrna, ['Name'], 'Description')
GeneExpressionData = GeneExpressionData.reset_index()
GeneExpressionData = GeneExpressionData.rename(columns={'index': 'Sample Identifier'})
GeneExpressionData = GeneExpressionData[GeneExpressionData['Sample Identifier'].isin(filter_ids)]

MutationData = create_binary_matrix(data_mutations, 'Tumor_Sample_Barcode', 'Hugo_Symbol')
MutationData = MutationData.rename(columns={'Sample_SNP': 'Sample Identifier'})
MutationData = MutationData[MutationData['Sample Identifier'].isin(filter_ids)]

CNA_Data = process_and_rename_columns(data_cna, 'Gene Symbol', False)
CNA_Data = CNA_Data.reset_index()
CNA_Data = CNA_Data.rename(columns={'index': 'Sample Identifier'})
CNA_Data = CNA_Data[CNA_Data['Sample Identifier'].isin(filter_ids)]

In [31]:
GeneExpressionData

Sample,Sample Identifier,WASH7P,RP11-34P13.15,RP11-34P13.16,RP11-34P13.13,RP11-34P13.18,AP006222.2,RPL23AP21,RPL23AP24,RP4-669L17.10,...,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB
1,SU2CLC-CLE-NIVO181,13.22710,16.02570,62.92200,0.030048,38.10830,0.322607,1.629270,1.450480,0.233069,...,13.607000,12.589500,15.853700,9.497160,8.069840,1.880250,23.09920,12.019300,10.99140,9.788480
2,SU2CLC-CLE-NIVO191,17.22610,14.78460,56.11330,0.011332,34.09670,0.112979,0.614472,0.273520,0.241727,...,31.150200,34.423400,25.720600,32.728800,7.405850,13.355200,42.59080,22.180700,27.54650,29.625700
7,SU2CLC-CLE-NIVO31,9.40064,5.54303,16.78250,0.004691,33.59760,0.122323,0.890314,0.981328,5.719080,...,49.584500,61.213600,67.513500,32.843500,9.911460,17.026500,74.23780,40.369800,51.92480,39.429100
10,SU2CLC-CLE-NIVO51,3.28665,8.04259,46.90420,0.006126,7.17184,0.056377,0.332175,0.394296,0.285108,...,3.384520,3.300090,3.845250,4.259810,2.851810,1.277810,8.70284,5.277950,7.15647,2.860460
16,SU2CLC-CLE-NIVO91,3.54148,19.24610,90.78630,0.078662,39.76650,0.098715,0.581635,0.805477,1.294280,...,54.923200,62.920600,80.470800,50.008400,19.974000,23.567700,89.66340,58.677100,63.54060,42.825800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,SU2CLC-MSK-A20131,9.95161,7.18469,24.84190,0.026859,42.42630,0.140069,1.019480,1.296580,2.562580,...,54.832100,56.268500,71.884100,42.362500,31.836100,32.718500,98.24240,70.194000,83.73570,44.741100
143,SU2CLC-MSK-A20141,1.31537,1.06024,4.62249,0.005169,1.69608,0.067384,0.490448,0.457417,0.066818,...,1.263920,0.309364,0.611233,0.694294,0.647788,0.539044,1.37092,0.751003,0.60989,0.561248
144,SU2CLC-MSK-A20601,1.37496,10.72470,11.40440,0.007688,6.13702,0.005896,0.000000,0.000000,0.357822,...,4.700340,5.982530,4.651110,7.016940,7.433600,1.683890,5.11557,4.718330,7.43937,3.798720
146,SU2CLC-UCD-11241,6.45020,13.91830,0.00000,0.010343,36.46370,0.087253,0.560833,0.665716,0.113656,...,0.421547,0.000000,0.141135,0.449507,0.370377,0.000000,2.02269,0.601147,0.91536,0.224628


In [32]:
MutationData

,Sample Identifier,MFN2_SNP,CROCC_SNP,RHD_SNP,TRNAU1AP_SNP,RBBP4_SNP,FLG_SNP,CLK2_SNP,GPR137B_SNP,WDR92_SNP,...,DNAJC15_SNP,MIR520F_SNP,IGLC3_SNP,SLMO2_SNP,HIST3H2BB_SNP,FHL2_SNP,HIST1H2BE_SNP,CHPT1_SNP,LRRC28_SNP,APOBEC3G_SNP
0,SU2CLC-CLE-NIVO181,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,SU2CLC-CLE-NIVO191,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SU2CLC-CLE-NIVO11,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SU2CLC-CLE-NIVO31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SU2CLC-CLE-NIVO41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-0101,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,SU2CLC-YAL-1031,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,SU2CLC-YAL-1041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,SU2CLC-YAL-1081,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0


In [33]:
CNA_Data

,Sample Identifier,ACAP3_CN,ACTRT2_CN,AGRN_CN,ANKRD65_CN,ATAD3A_CN,ATAD3B_CN,ATAD3C_CN,AURKAIP1_CN,B3GALT6_CN,...,RN7SL500P_CN,SBF1_CN,SCO2_CN,SHANK3_CN,SYCE3_CN,TRABD_CN,TTLL8_CN,TUBGCP6_CN,TYMP_CN,ZBED4_CN
0,SU2CLC-UCD-11241,-0.031,-0.031,-0.031,-0.031,-0.031,-0.031,-0.031,-0.031,-0.031,...,-0.393,-0.393,-0.393,-0.393,-0.393,-0.393,-0.393,-0.393,-0.393,-0.393
1,SU2CLC-UCD-11451,0.244,0.244,0.244,0.244,0.244,0.244,0.244,0.244,0.244,...,-0.286,-0.286,-0.286,-0.286,-0.286,-0.286,-0.286,-0.286,-0.286,-0.286
2,SU2CLC-UCD-11421,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,...,-0.201,-0.201,-0.201,-0.201,-0.201,-0.201,-0.201,-0.201,-0.201,-0.201
3,SU2CLC-COL-10011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.117,-0.117,-0.117,-0.117,-0.117,-0.117,-0.117,-0.117,-0.117,-0.117
4,SU2CLC-MSK-A20011,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,...,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-1041,0.127,0.127,0.127,0.127,0.127,0.127,0.127,0.127,0.127,...,-0.044,-0.044,-0.044,-0.044,-0.044,-0.044,-0.044,-0.044,-0.044,-0.044
305,SU2CLC-YAL-1081,-0.731,-0.731,-0.731,-0.731,-0.731,-0.731,-0.731,-0.731,-0.731,...,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09
306,SU2CLC-YAL-1111,0.096,0.096,0.096,0.096,0.096,0.096,0.096,0.096,0.096,...,-0.267,-0.267,-0.267,-0.267,-0.267,-0.267,-0.267,-0.267,-0.267,-0.267
307,SU2CLC-MSK-A21031,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,-0.009,-0.009,-0.009,-0.009,-0.009,-0.009,-0.009,-0.009,-0.009,-0.009


# **Feature Selection**

In [34]:
'''
variable                      rows                Columns
GeneExpressionData            Samples             Genes
ProcessedClinicalData         Samples             Clincal data
CNA_Data                      Samples             Genes
MutationData                  Samples             Genes
labels                        Samples             ICB Response
CibersortData                 Samples             Immune cells
'''

'\nvariable                      rows                Columns\nGeneExpressionData            Samples             Genes\nProcessedClinicalData         Samples             Clincal data\nCNA_Data                      Samples             Genes\nMutationData                  Samples             Genes\nlabels                        Samples             ICB Response\nCibersortData                 Samples             Immune cells\n'

## **Random Forest**

In [37]:
'''
A function that implements feature selection with random forest
It builds a RF model and retrieves the feature importances
Then we sort by importance in descending order and get the top 1000 genes on the condition that they are larger than 0.
It returns a random forest importance dataframe with two columns: gene and importance

Note: this same sequence is almost the same in all the feature selection methods, so we will not write more comments in further similar functions
'''

def feature_selection_with_rf(df, label_df, commoncolumn='Sample Identifier', labelID ='ICB Response', n_feature=1000, n_estimators=100, random_state=42):
    merged_df = pd.merge(df, label_df, on=commoncolumn)
    X = merged_df.drop(columns=[commoncolumn, labelID])
    y = merged_df[labelID]
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X, y)
    rf_importances = rf.feature_importances_
    rf_importances_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_importances
    })
    rf_importances_df = rf_importances_df.sort_values(by='Importance', ascending=False)
    rf_importances_df = rf_importances_df.head(n_feature)
    rf_importances_df = rf_importances_df[rf_importances_df['Importance'] > 0]
    return rf_importances_df

In [39]:
# Selecting genes (expression, SNP mutation, and CNA mutation) with the random forest method
rf_Expression = feature_selection_with_rf(GeneExpressionData, labels, 'Sample Identifier', 'ICB Response', n_estimators=1000)
rf_SNP = feature_selection_with_rf(MutationData, labels, 'Sample Identifier', 'ICB Response',  n_estimators=1000)
rf_CNA = feature_selection_with_rf(CNA_Data, labels, 'Sample Identifier', 'ICB Response',  n_estimators=1000)

In [40]:
rf_Expression

,Feature,Importance
714,LURAP1,0.002704
18798,TM6SF2,0.002233
16343,ZDHHC1,0.001708
14210,UBE2CP1,0.001670
1810,RP11-142L4.3,0.001642
...,...,...
9819,FAM166B,0.000302
12797,PIK3C2G,0.000302
6758,GCNT2,0.000302
6863,HIST1H2BI,0.000302


In [41]:
# Selecting a certain number of immune cells

CibersortData = CibersortData.rename(columns={'Mixture': 'Sample Identifier'})
SelectedCibersortCells = feature_selection_with_rf(CibersortData, labels, 'Sample Identifier', 'ICB Response', n_feature = 9,  n_estimators=10)
# these 3 steps add the column Sample Identifies to be at the beginning :D
columnstokeep = list(SelectedCibersortCells['Feature'])
columnstokeep.append('Sample Identifier')
columnstokeep.insert(0, columnstokeep.pop())

FinalCibersortData = CibersortData[columnstokeep]
FinalCibersortData = FinalCibersortData[FinalCibersortData['Sample Identifier'].isin(filter_ids)]

In [42]:
FinalCibersortData

,Sample Identifier,Neutrophils,Macrophages M1,Monocytes,T cells follicular helper,B cells naive,T cells CD4 memory activated,T cells CD8,NK cells activated,Macrophages M0
1,SU2CLC-CLE-NIVO181,0.013690,0.035234,0.024509,0.062979,0.198873,0.066236,0.177521,0.027368,0.194549
2,SU2CLC-CLE-NIVO191,0.001007,0.064350,0.005999,0.098456,0.201215,0.052820,0.101970,0.021795,0.158986
7,SU2CLC-CLE-NIVO31,0.000635,0.013140,0.000000,0.041894,0.111251,0.000000,0.050663,0.070540,0.079434
10,SU2CLC-CLE-NIVO51,0.000000,0.092092,0.020477,0.100630,0.254994,0.003447,0.150072,0.060296,0.046533
16,SU2CLC-CLE-NIVO91,0.009611,0.018045,0.069118,0.038607,0.033851,0.000000,0.075424,0.000000,0.156832
...,...,...,...,...,...,...,...,...,...,...
142,SU2CLC-MSK-A20131,0.001322,0.035721,0.005841,0.098243,0.323012,0.046111,0.116798,0.000000,0.183699
143,SU2CLC-MSK-A20141,0.019880,0.035289,0.018134,0.048395,0.162913,0.006579,0.075006,0.060067,0.128707
144,SU2CLC-MSK-A20601,0.049670,0.108429,0.017177,0.148289,0.065879,0.097346,0.209832,0.066949,0.186908
146,SU2CLC-UCD-11241,0.000000,0.007278,0.041408,0.084042,0.139079,0.000000,0.111823,0.063486,0.237776


## **Recursive Feature Elimination**

In [63]:
def recursive_feature_elimination(df, labels_df, commoncolumn='Sample Identifier', labelID ='ICB Response', n_features_to_select=1000, step=100):
    merged_df = pd.merge(df, labels_df, on=commoncolumn)
    X = merged_df.drop([commoncolumn, labelID], axis=1)
    y = merged_df[labelID]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    logreg = LogisticRegression()
    rfe = RFE(estimator=logreg, n_features_to_select=n_features_to_select, step=step)
    rfe.fit(X_scaled, y)
    selected_features = pd.DataFrame({'Feature': X.columns,
                                      'Selected': rfe.support_,
                                      'Ranking': rfe.ranking_})
    s = selected_features.sort_values(by='Ranking', ascending=True)

    # Selecting genes with ranking = 1, that is the top n_features_to_select genes
    s = s[s['Ranking'] == 1]
    return s

In [64]:
rfe_Expression = recursive_feature_elimination(GeneExpressionData, labels, 'Sample Identifier', 'ICB Response', step=50)
rfe_SNP = recursive_feature_elimination(MutationData, labels, 'Sample Identifier', 'ICB Response', step=50)
rfe_CNA = recursive_feature_elimination(CNA_Data, labels, 'Sample Identifier', 'ICB Response', step=250)

In [65]:
rfe_Expression

,Feature,Selected,Ranking
8890,NUB1,True,1
15352,C15orf59,True,1
15348,HCN4,True,1
19648,C19orf18,True,1
3928,ACKR2,True,1
...,...,...,...
2576,AC010733.5,True,1
6761,C6orf52,True,1
2638,B3GALNT1P1,True,1
3598,UGT1A5,True,1


In [ ]:
# rfe_CNA.to_csv('rfe_CNA.tsv', sep='\t', index=False) # bec. it takes so much time, so we already ran it and saved the results

## **SelectFromModel**

In [46]:
def select_from_model(df, labels_df, commoncolumn='Sample Identifier', labelID ='ICB Response'):
# Selecting features using Logistic Regression and SelectFromModel
  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]
  log_reg = LogisticRegression()
  select_model = SelectFromModel(log_reg)
  select_model.fit(X, y)
  selection_status = select_model.get_support()
  SelectFromModel_genes_df = pd.DataFrame({
      'Feature': X.columns,
      'Selected': selection_status
  })
  SelectFromModel_genes_df['Selected'] = SelectFromModel_genes_df['Selected'].astype(int)
  SelectFromModel_genes_df = SelectFromModel_genes_df[SelectFromModel_genes_df['Selected'] == 1]

  return SelectFromModel_genes_df

In [ ]:
sfm_Expression = select_from_model(GeneExpressionData, labels, 'Sample Identifier', 'ICB Response')
sfm_SNP = select_from_model(MutationData, labels, 'Sample Identifier', 'ICB Response')
sfm_CNA = select_from_model(CNA_Data, labels, 'Sample Identifier', 'ICB Response')

In [48]:
sfm_Expression

,Feature,Selected
1,RP11-34P13.15,1
2,RP11-34P13.16,1
17,NOC2L,1
24,AGRN,1
32,SDF4,1
...,...,...
21720,TXLNGY,1
21721,KDM5D,1
21722,MT-RNR1,1
21723,MT-RNR2,1


## **Correlation**

In [51]:
def correlation(df, labels_df, threshold=0.2, commoncolumn='Sample Identifier', labelID ='ICB Response', is_df_categorical=True):
  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]

  # Was producing an error because Pearson correlation (calculated by corr function) cannot correlate categoricals to floats
  # So we added an else condition to handle the case where the data is not categorical to be calculated via point biserial method
  if is_df_categorical:
    correlation_values = X.apply(lambda gene: gene.corr(y))
  else:
    correlation_values = X.apply(lambda gene: stats.pointbiserialr(gene, y).correlation)


  gene_correlation = pd.DataFrame({
      'Feature': correlation_values.index,
      'Correlation': correlation_values
  })

  gene_correlation = gene_correlation.sort_values(by='Correlation', key=abs, ascending=False)

  selected_genes = gene_correlation[abs(gene_correlation['Correlation']) > threshold]

  return selected_genes

In [52]:
Expression_correlation = correlation(GeneExpressionData, labels, 0.2, commoncolumn='Sample Identifier', labelID ='ICB Response')
SNP_correlation = correlation(MutationData, labels, 0.15, commoncolumn='Sample Identifier', labelID ='ICB Response')
CNA_correlation = correlation(CNA_Data, labels, 0.15, commoncolumn='Sample Identifier', labelID ='ICB Response')

In [53]:
Expression_correlation

,Feature,Correlation
RP11-364L4.1,RP11-364L4.1,-0.411084
WARS,WARS,0.393032
UBD,UBD,0.392057
ARHGEF37,ARHGEF37,-0.389179
RP11-274E7.2,RP11-274E7.2,0.378198
...,...,...
SGCD,SGCD,-0.200102
LYPD5,LYPD5,0.200101
RLBP1,RLBP1,-0.200079
FAM57A,FAM57A,-0.200050


## **Gradient Boosting**

In [54]:
def gradient_boosting(df, labels_df, commoncolumn='Sample Identifier', labelID ='ICB Response', nofeatures=1000, n_estimator = 100, random_state=42):
  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]
  gb_classifier = GradientBoostingClassifier(n_estimators=n_estimator, random_state=random_state)

  gb_classifier.fit(X, y)

  feature_importances_gb = gb_classifier.feature_importances_

  gene_importances_gb = pd.DataFrame({
      'Feature': X.columns,
      'Importance': feature_importances_gb
  })

  sorted_gene_importances_gb = gene_importances_gb.sort_values(by='Importance', ascending=False)
  sorted_gene_importances_gb = sorted_gene_importances_gb.head(nofeatures)
  sorted_gene_importances_gb = sorted_gene_importances_gb[sorted_gene_importances_gb['Importance'] > 0]
  return sorted_gene_importances_gb


In [55]:
gb_Expression = gradient_boosting(GeneExpressionData, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')
gb_SNP = gradient_boosting(MutationData, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')
gb_CNA = gradient_boosting(CNA_Data, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')

In [56]:
gb_Expression

,Feature,Importance
714,LURAP1,3.563446e-01
50,ANKRD65,2.372818e-01
7571,RNF217,1.975264e-01
7689,YAP1P1,1.803312e-02
9925,SMC5-AS1,1.461033e-02
...,...,...
20317,KCNE1,3.371244e-11
6796,RPL7P26,3.197956e-11
20965,NCAPH2,2.760140e-11
12563,CACNA1C,2.618264e-11


## **FDR**

In [68]:
def fdr(df, labels_df, commoncolumn='Sample Identifier', n_feature=1000, labelID ='ICB Response'):
  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]

  f_scores, _ = f_classif(X, y)

  fdr_df = pd.DataFrame({'Feature': X.columns, 'Fisher_Score': f_scores})

  fdr_df = fdr_df.sort_values(by='Fisher_Score', ascending=False)

  fdr_df = fdr_df.head(n_feature)

  return fdr_df

In [69]:
fdr_Expression = fdr(GeneExpressionData, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')
fdr_SNP = fdr(MutationData, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')
fdr_CNA = fdr(CNA_Data, labels, commoncolumn='Sample Identifier', labelID ='ICB Response')

In [70]:
fdr_Expression

,Feature,Fisher_Score
5513,RP11-364L4.1,12.811349
14691,WARS,11.509828
6920,UBD,11.442433
6458,ARHGEF37,11.245191
6114,RP11-274E7.2,10.515177
...,...,...
19802,DSTN,3.513181
16771,DVL2,3.512626
233,FBXO42,3.512491
19707,SIRPB1,3.511991


## **Intersection Selection Methods**

In [ ]:

'''
random forest --> rf_Expression
recursive feature elimination --> rfe_Expression
Select from model --> sfm_Expression
correlation --> Expression_correlation
gradient boosting --> gb_Expression
fdr --> fdr_Expression

and so on for SNP and CNA
'''

In [71]:
# Code only made to visualize the resulting dataframe, but it will not be used later

fisher_selected = set(fdr_Expression['Feature']) #if we want SNP, we should make it fdr_SNP instead and so on

gradient_boosting_selected = set(gb_Expression['Feature'])

correlation_selected = set(Expression_correlation['Feature'])

sfm_selected = set(sfm_Expression['Feature'])

random_forest_selected = set(rf_Expression['Feature'])

rfe_selected = set(rfe_Expression['Feature'])

all_genes = set.union(fisher_selected, gradient_boosting_selected, correlation_selected, sfm_selected, random_forest_selected, rfe_selected)

all_genes_list = list(all_genes)

final_df = pd.DataFrame(index=all_genes_list, columns=["Fisher", "Gradient Boosting", "Correlation", "Select From Model", "Random Forest", "Recursive Feature Elimination"])

final_df["Fisher"] = final_df.index.isin(fisher_selected).astype(int)
final_df["Gradient Boosting"] = final_df.index.isin(gradient_boosting_selected).astype(int)
final_df["Correlation"] = final_df.index.isin(correlation_selected).astype(int)
final_df["Select From Model"] = final_df.index.isin(sfm_selected).astype(int)
final_df["Random Forest"] = final_df.index.isin(random_forest_selected).astype(int)
final_df["Recursive Feature Elimination"] = final_df.index.isin(rfe_selected).astype(int)

In [72]:
final_df

,Fisher,Gradient Boosting,Correlation,Select From Model,Random Forest,Recursive Feature Elimination
WWP1,0,0,0,1,0,0
ANLN,0,0,1,1,0,0
BSCL2,0,0,0,1,0,0
QSOX1,0,0,0,1,0,0
HS6ST2,0,0,0,0,0,1
...,...,...,...,...,...,...
TAS2R40,1,0,1,0,0,1
SNORA20,0,0,0,1,0,0
MAST4,1,0,1,0,0,0
HMOX1,0,0,0,1,0,0


In [73]:
Merged_Expression = get_merged_df(rf_Expression, fdr_Expression, Expression_correlation, sfm_Expression, gb_Expression, rfe_Expression)
Merged_SNP = get_merged_df(rf_SNP, fdr_SNP, SNP_correlation, sfm_SNP, gb_SNP, rfe_SNP)
Merged_CNA = get_merged_df(rf_CNA, fdr_CNA, CNA_correlation, sfm_CNA, gb_CNA, rfe_CNA)

In [74]:
Merged_Expression

,Random_Forest,Select_From_Model,Gene_Response_Correlation,Gradient_Boosting,Fisher_Discriminant_Ratio,Recursive_Feature_Elimination
WWP1,NaN,WWP1,NaN,NaN,NaN,NaN
ANLN,NaN,ANLN,ANLN,NaN,NaN,NaN
BSCL2,NaN,BSCL2,NaN,NaN,NaN,NaN
QSOX1,NaN,QSOX1,NaN,NaN,NaN,NaN
HS6ST2,NaN,NaN,NaN,NaN,NaN,HS6ST2
...,...,...,...,...,...,...
TAS2R40,NaN,NaN,TAS2R40,NaN,TAS2R40,TAS2R40
SNORA20,NaN,SNORA20,NaN,NaN,NaN,NaN
MAST4,NaN,NaN,MAST4,NaN,MAST4,NaN
HMOX1,NaN,HMOX1,NaN,NaN,NaN,NaN


In [75]:
Best_Expression_Genes = Best_Combinations(Merged_Expression)
Best_SNP_Genes = Best_Combinations(Merged_SNP)
Best_CNA_Genes = Best_Combinations(Merged_CNA)

In [76]:
Best_Expression_Genes

['MIR612',
 'PPIAP22',
 'TCF7L1',
 'NUB1',
 'PNPLA2',
 'RPL7',
 'EGR1',
 'RPSAP58',
 'SLC44A3',
 'HIST1H2AJ',
 'RPL13AP6',
 'IDO1',
 'LARGE2',
 'SCARA3',
 'JUNB',
 'IL2RG',
 'HIST1H3C',
 'CXCL13',
 'PDZD2',
 'FLG',
 'DDX3Y',
 'GBP4',
 'SLC2A3',
 'SNORA63',
 'XDH',
 'TRDC',
 'HSPB8',
 'RP11-44K6.4',
 'NAA10',
 'SCARNA3',
 'NPR1',
 'SLC11A2',
 'NLGN4Y',
 'SYTL3',
 'SDF4',
 'PSME2',
 'ACSS2',
 'IGHV1-18',
 'CELSR2',
 'STAT1',
 'NUPL2',
 'S100A4',
 'RP11-843B15.2',
 'WARS',
 'APOL1',
 'DDX43',
 'XXbac-BPG248L24.12',
 'PTMAP5',
 'EEF1A1P6',
 'UBTD1',
 'HIST1H4I',
 'PLS3']

In [77]:
# Only to save these best genes

best_genes_expression = pd.DataFrame(Best_Expression_Genes, columns=['Gene_Name'])
best_genes_SNP = pd.DataFrame(Best_SNP_Genes, columns=['Gene_Name'])
best_genes_CNA = pd.DataFrame(Best_CNA_Genes, columns=['Gene_Name'])

best_genes_expression.to_csv('Best Expression Genes.tsv', sep='\t', index=False)
best_genes_SNP.to_csv('Best SNP Genes.tsv', sep='\t', index=False)
best_genes_CNA.to_csv('Best CNA Genes.tsv', sep='\t', index=False)

In [79]:
CNA_Data_selected = [gene + '_CN' for gene in Best_Expression_Genes if gene + '_CN' in CNA_Data.columns]
MutationData_selected = [gene + '_SNP' for gene in Best_Expression_Genes if gene + '_SNP' in MutationData.columns]

# These are the filtered CNA and SNP mutation data, filtered by their own feature selection methods.

filtered_CNA_Data = CNA_Data[['Sample Identifier'] + Best_CNA_Genes]
filtered_SNP_Data = MutationData[['Sample Identifier'] + Best_SNP_Genes]

filtered_Expression_Data = GeneExpressionData[['Sample Identifier'] + Best_Expression_Genes]

# These are the best_expression_genes-filtered CNA and SNP mutation data.
expression_filtered_CNA_Data = CNA_Data[['Sample Identifier'] + CNA_Data_selected]
expression_filtered_SNP_Data = MutationData[['Sample Identifier'] + MutationData_selected]

In [80]:
filtered_Expression_Data

Sample,Sample Identifier,MIR612,PPIAP22,TCF7L1,NUB1,PNPLA2,RPL7,EGR1,RPSAP58,SLC44A3,...,RP11-843B15.2,WARS,APOL1,DDX43,XXbac-BPG248L24.12,PTMAP5,EEF1A1P6,UBTD1,HIST1H4I,PLS3
1,SU2CLC-CLE-NIVO181,348.0900,24.29600,3.78420,31.6767,43.6228,129.188,89.7503,20.96220,24.59000,...,19.66640,25.52880,17.1987,1.747150,309.740,18.26200,4.70741,2.94601,3.74015,47.8990
2,SU2CLC-CLE-NIVO191,406.1270,66.11530,14.00090,98.6591,63.7122,403.858,220.9910,26.20770,23.17280,...,38.37900,113.11000,31.9941,1.331590,288.854,21.99140,3.04350,9.95852,5.94071,272.6860
7,SU2CLC-CLE-NIVO31,630.3640,39.68360,5.43705,99.3546,62.5895,132.664,225.7480,39.01170,8.55968,...,39.91620,26.59100,15.0406,33.824700,530.513,38.29130,2.66685,3.49226,4.22236,31.9941
10,SU2CLC-CLE-NIVO51,229.2240,29.87310,8.00819,46.5119,87.9062,285.782,130.4340,4.57292,39.69710,...,9.32457,33.82420,123.6060,1.276410,231.344,67.14910,7.76021,1.64617,1.73038,162.0350
16,SU2CLC-CLE-NIVO91,680.0240,34.24890,20.73700,49.0811,189.6120,228.471,522.1140,11.92340,48.02260,...,44.30120,20.24700,23.3483,10.152700,213.638,15.97440,3.42502,17.86590,4.27949,172.4920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,SU2CLC-MSK-A20131,200.3380,19.17870,21.76910,51.9318,90.8587,172.965,57.6175,7.72008,12.74780,...,30.49870,21.54360,43.9367,1.861110,267.976,9.13591,4.23198,8.97318,5.91508,191.0840
143,SU2CLC-MSK-A20141,16.8101,7.61903,3.76963,11.6312,35.9227,927.459,48.4013,3.13702,8.53214,...,7.39510,65.00590,73.1467,0.475843,255.653,7.88081,1.12206,5.49920,5.41902,72.9429
144,SU2CLC-MSK-A20601,14.5272,5.21253,5.30722,33.3292,20.2938,147.593,21.3713,30.78790,6.99759,...,9.83030,103.95100,34.2863,0.083823,954.525,6.33292,1.44542,2.86172,22.37950,132.0290
146,SU2CLC-UCD-11241,1263.8800,129.75900,31.48520,39.9719,129.5620,176.698,259.6800,76.41400,8.47561,...,7.87164,9.92371,34.4552,2.418170,219.285,232.36900,51.45920,18.04700,17.99950,795.6870


# **Classifiers**

In [ ]:
'''
our selected snp genes --> Best_SNP_Genes
our selected cn genes --> Best_CNA_Genes
our selected EXPRESSION genes --> Best_Expression_Genes


final immune cell abundance --> FinalCibersortData
final clinical --> ProcessedClinicalData
final cna --> filtered_CNA_Data
final snp --> filtered_SNP_Data
final expression --> filtered_Expression_Data
final expression-filtered cna --> expression_filtered_CNA_Data
final expression-filtered snp --> expression_filtered_SNP_Data
'''

## **Random Forest**

In [83]:
'''
A function for building an optimized random forest model. All it needs is a dataframe for the features and another one for the labels
It returns a dataframe that contains all the results (accuracies)
'''

def randomforest(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    param_grid = {
        'n_estimators': [1, 10, 50, 100, 1000],
        'max_depth': [4, 5, 6, 7, 8],
        'criterion': ['gini', 'entropy']
    }
    clf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_clf = grid_search.best_estimator_
    y_pred = best_clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    results = {
        'Mean CV Accuracy': grid_search.best_score_,
        'Test Accuracy': test_accuracy,
        'F1 Score': f1,
        'AUC Accuracy': roc_auc
    }
    # print("Best Parameters:", grid_search.best_params_)
    return results

## **Support Vector Machine**

In [88]:
def svm_classifier(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    param_grid = {
        'C': [0.1, 1, 10, 100],  # Regularization parameter
        'gamma': ['scale', 'auto'],  # Kernel coefficient
        'kernel': ['linear', 'rbf', 'poly']  # Specifies the kernel type
    }

    clf = SVC(probability=True, random_state=42)  # probability=True is needed for predict_proba
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_clf = grid_search.best_estimator_
    y_pred = best_clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results = {
        'Mean CV Accuracy': grid_search.best_score_,
        'Test Accuracy': test_accuracy,
        'F1 Score': f1,
        'AUC Accuracy': roc_auc
    }
    print("Best Parameters:", grid_search.best_params_)

    return results

## **Logistic Regression**

In [91]:
def logistic_regression_classifier(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']  # Algorithm to use in the optimization problem
    }

    clf = LogisticRegression(random_state=42, max_iter=1000)
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_clf = grid_search.best_estimator_
    y_pred = best_clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results = {
        'Mean CV Accuracy': grid_search.best_score_,
        'Test Accuracy': test_accuracy,
        'F1 Score': f1,
        'AUC Accuracy': roc_auc
    }
    print("Best Parameters:", grid_search.best_params_)

    return results

## **A model with RF & PCA feature selection, just as a baseline to compare our results to**

In [86]:
results = []

pca_model = merge_dataframes([GeneExpressionData, labels], 'Sample Identifier') # change to any desired model

X = pca_model.iloc[:, 1:-1]

y = pca_model.iloc[:, -1]

n_components_options = [40, 25, 15, 10]

for n_components in n_components_options:
    pca = PCA(n_components=n_components)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    classifier = RandomForestClassifier(random_state=0, n_estimators=10)

    pipeline = make_pipeline(pca, classifier)

    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)

    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

    roc_auc = roc_auc_score(y_test, y_pred_proba)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='binary')

    results.append({
        'n_components': n_components,
        'CV_Mean_Accuracy': np.mean(cv_scores),
        'Test_Accuracy': accuracy,
        'Test_F1_Score': f1,
        'AUC-ROC Score': roc_auc
    })

results_df = pd.DataFrame(results)
results_df

,n_components,CV_Mean_Accuracy,Test_Accuracy,Test_F1_Score,AUC-ROC Score
0,40,0.463636,0.461538,0.222222,0.404762
1,25,0.403636,0.615385,0.545455,0.761905
2,15,0.521818,0.384615,0.428571,0.357143
3,10,0.505455,0.461538,0.461538,0.488095


## **A model with RF feature selection, just as a baseline to compare our results to**

In [87]:
results = []

model = merge_dataframes([GeneExpressionData, labels], 'Sample Identifier') # change to any desired model

X = model.iloc[:, 1:-1]

y = model.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

initial_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
initial_clf.fit(X_train, y_train)

importances = initial_clf.feature_importances_

indices = np.argsort(importances)[::-1][:40] # Top n features
X_train_reduced = X_train.iloc[:, indices]
X_test_reduced = X_test.iloc[:, indices]

clf = RandomForestClassifier(n_estimators=1000, random_state=42)

cv_scores = cross_val_score(clf, X_train_reduced, y_train, cv=5, scoring='accuracy')
mean_cv_accuracy = np.mean(cv_scores)

clf.fit(X_train_reduced, y_train)

y_pred = clf.predict(X_test_reduced)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
y_pred_proba = clf.predict_proba(X_test_reduced)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Mean CV Accuracy: {mean_cv_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"AUC-ROC Score: {roc_auc:.2f}")

Mean CV Accuracy: 0.90
Test Accuracy: 0.54
F1 Score: 0.57
AUC-ROC Score: 0.45


## **For quickly building a simple model from our filtered dataframes.**

In [82]:
model = merge_dataframes([filtered_Expression_Data, labels], 'Sample Identifier') # add here the desired dataframes (set of features)
mRNA_model_rf = randomforest(model.iloc[:, 1:-1], model.iloc[:, -1]) # change "randomforest" to any other classifier as desired
mRNA_model_df = pd.DataFrame([mRNA_model_rf])
mRNA_model_df

Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 50}


,Mean CV Accuracy,Test Accuracy,F1 Score,AUC Accuracy
0,0.830909,0.769231,0.769231,0.8375


## **Model Combinations**

In [ ]:
'''
The block of code that does wonders :'D --> It tries all the different combination from all the "features" dataframes
Note: It takes quite some time to run
'''

dfs = {
    'filtered_SNP_Data': expression_filtered_SNP_Data, # can be changed to filtered_SNP_Data
    'filtered_CNA_Data': expression_filtered_CNA_Data, # can be changed to filtered_CNA_Data
    'filtered_mRNA_Data': filtered_Expression_Data,
    'ProcessedClinicalData': ProcessedClinicalData,
    'FinalCibersortData': FinalCibersortData
}

all_results = []
for r in range(1, len(dfs) + 1):
    for combo_names in itertools.combinations(dfs.keys(), r):
        combo_dfs = [dfs[name] for name in combo_names]
        merged_df = merge_dataframes(combo_dfs + [labels], 'Sample Identifier')
        results = randomforest(merged_df.iloc[:, 1:-1], merged_df.iloc[:, -1]) # change "randomforest" to any other classifier as desired
        results['Combination'] = ' + '.join(combo_names)
        all_results.append(results)

all_results_df = pd.DataFrame(all_results)

In [ ]:
# all_results_df.to_csv('Random_Forest_Results_Five_Combinations_Expression_Related_To_Mutations.tsv', sep='\t', index=False) # bec. it takes so much time, so we already ran it and saved the results
# and so on for the rest.. we already ran them

# **TRASH --> Useless Code (Don't Run it)**

## **RF Model on Mutations Data**

In [ ]:
# List to store results
results = []

X_mut = mutation_and_clinical_and_cibersort_labelled.iloc[:, :-1]
X_mut = X_mut.iloc[:, 1:]
y_mut = mutation_and_clinical_and_cibersort_labelled.iloc[:, -1]

# Dimensionality reduction options
n_components_options = [75, 62, 50, 24, 10]

for n_components in n_components_options:
    # Apply PCA
    pca = PCA(n_components=n_components)

    # Splitting the dataset into the Training set and Test set
    X_mut_train, X_mut_test, y_mut_train, y_mut_test = train_test_split(X_mut, y_mut, test_size=0.2, random_state=0)

    # Initialize Random Forest Classifier
    classifier = RandomForestClassifier(random_state=0, n_estimators = 100)

    # Create a pipeline
    pipeline = make_pipeline(pca, classifier)

    # Cross-validation
    cv_scores = cross_val_score(pipeline, X_mut_train, y_mut_train, cv=5)

    # Train the classifier with the training set
    pipeline.fit(X_mut_train, y_mut_train)

    # Predict the Test set results
    y_pred = pipeline.predict(X_mut_test)

    # Evaluate the model
    accuracy = accuracy_score(y_mut_test, y_pred)
    f1 = f1_score(y_mut_test, y_pred, average='binary')  # Use 'micro', 'macro', or 'weighted' for multi-class

    # Store results
    results.append({
        'n_components': n_components,
        'CV_Mean_Accuracy': np.mean(cv_scores),
        'Test_Accuracy': accuracy,
        'Test_F1_Score': f1
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

In [ ]:
results_df

,n_components,CV_Mean_Accuracy,Test_Accuracy,Test_F1_Score
0,75,0.642105,0.541667,0.153846
1,62,0.610526,0.500000,0.000000
2,50,0.589474,0.583333,0.500000
3,24,0.589474,0.583333,0.375000
4,10,0.621053,0.583333,0.444444


## **Preparing the model**

In [ ]:
# Use the intersection of DataFrame columns and gene list to filter
filtered_genes = set(Final_Selected_Genes)
searching_genes_snp = mutation_and_clinical_and_cibersort_labelled.columns.intersection(filtered_genes).values
searching_genes_cna = cna_mutations_and_clinical_and_cibersort_labelled.columns.intersection(filtered_genes).values

# Create a new DataFrame with the filtered columns
clinical_and_immune_cols = mutation_and_clinical_and_cibersort_labelled.iloc[:,-39:-2].columns.values

In [ ]:
final_cols_snp = searching_genes_snp.tolist() + clinical_and_immune_cols.tolist()
final_cols_cna = searching_genes_cna.tolist() + clinical_and_immune_cols.tolist()
final_cols_snp.append('ICB Response')
final_cols_cna.append('ICB Response')

In [ ]:
model_snp = mutation_and_clinical_and_cibersort_labelled[final_cols_snp]
model_cna = cna_mutations_and_clinical_and_cibersort_labelled[final_cols_cna]

## **Running the model**

In [ ]:
results = {}

X_snp = model_snp.iloc[:, :-1]
y_snp = model_snp.iloc[:, -1]

# Splitting the dataset into the Training set and Test set
X_snp_train, X_snp_test, y_snp_train, y_snp_test = train_test_split(X_snp, y_snp, test_size=0.2, random_state=0)

# Initialize Random Forest Classifier
classifier = RandomForestClassifier(random_state=0, n_estimators= 100)

# Cross-validation
cv_scores = cross_val_score(classifier, X_snp_train, y_snp_train, cv=5)

# Train the classifier with the training set
classifier.fit(X_snp_train, y_snp_train)

# Predict the Test set results
y_pred = classifier.predict(X_snp_test)

# Evaluate the model
accuracy = accuracy_score(y_snp_test, y_pred)
f1 = f1_score(y_snp_test, y_pred, average='binary')

# Store results
results = {
    'CV_Mean_Accuracy': np.mean(cv_scores),
    'Test_Accuracy': accuracy,
    'Test_F1_Score': f1
}

## **TMB model**

In [ ]:
results = {}

X_tmb = model_snp['TMB']
y_tmb = model_snp.iloc[:, -1]

# Splitting the dataset into the Training set and Test set
X_snp_train, X_snp_test, y_snp_train, y_snp_test = train_test_split(X_snp, y_snp, test_size=0.2, random_state=0)

# Initialize Random Forest Classifier
classifier = RandomForestClassifier(random_state=0, n_estimators= 100)

# Cross-validation
cv_scores = cross_val_score(classifier, X_snp_train, y_snp_train, cv=5)

# Train the classifier with the training set
classifier.fit(X_snp_train, y_snp_train)

# Predict the Test set results
y_pred = classifier.predict(X_snp_test)

# Evaluate the model
accuracy = accuracy_score(y_snp_test, y_pred)
f1 = f1_score(y_snp_test, y_pred, average='binary')

# Store results
results = {
    'CV_Mean_Accuracy': np.mean(cv_scores),
    'Test_Accuracy': accuracy,
    'Test_F1_Score': f1
}

{'CV_Mean_Accuracy': 0.7263157894736842,
 'Test_Accuracy': 0.5,
 'Test_F1_Score': 0.14285714285714288}

In [ ]:
# # Get all column names except the first one (which seems to be an identifier column)
# column_names = merged_df.columns

# # Generate all combinations of four columns
# column_combinations = combinations(column_names, 4)

# # Dictionary to store the intersection results for each combination
# intersections = {}

# # Compute the intersection for each combination
# for combination in column_combinations:
#     # Drop NaN values and find the intersection
#     intersected_genes = set(merged_df[combination[0]].dropna()) & set(merged_df[combination[1]].dropna()) & \
#                         set(merged_df[combination[2]].dropna()) & set(merged_df[combination[3]].dropna())
#     # Store the result
#     intersections[combination] = intersected_genes

# # Displaying the results
# intersections_summary = {combo: len(genes) for combo, genes in intersections.items()}